In [5]:
import os
import boto3

from dotenv import load_dotenv


load_dotenv()
SPACES_KEY = os.getenv("SPACES_KEY")
SPACES_SECRET = os.getenv("SPACES_SECRET")



session = boto3.session.Session()
client = session.client('s3',
                        region_name='nyc3',
                        endpoint_url='https://nyc3.digitaloceanspaces.com',
                        aws_access_key_id=SPACES_KEY,
                        aws_secret_access_key=SPACES_SECRET)


response = client.list_buckets()
print(response)

spaces = [space['Name'] for space in response['Buckets']]
print("Spaces List: %s" % spaces)

print(response["Buckets"])



{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 25 Feb 2024 23:10:34 GMT', 'content-length': '318', 'content-type': 'text/xml; charset=utf-8', 'strict-transport-security': 'max-age=15552000; includeSubDomains; preload', 'x-envoy-upstream-healthchecked-cluster': ''}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'brbus-dataset', 'CreationDate': datetime.datetime(2024, 2, 20, 12, 47, 8, 928000, tzinfo=tzutc())}], 'Owner': {'DisplayName': '15685762', 'ID': '15685762'}}
Spaces List: ['brbus-dataset']
[{'Name': 'brbus-dataset', 'CreationDate': datetime.datetime(2024, 2, 20, 12, 47, 8, 928000, tzinfo=tzutc())}]


In [5]:
import os
from dotenv import load_dotenv


load_dotenv()
SPACES_KEY = os.getenv("SPACES_KEY")
SPACES_SECRET = os.getenv("SPACES_SECRET")



session = boto3.session.Session()
client = session.client('s3',
                        region_name='nyc3',
                        endpoint_url='https://nyc3.digitaloceanspaces.com',
                        aws_access_key_id=SPACES_KEY,
                        aws_secret_access_key=SPACES_SECRET)



# files_list = []
for city in ["rj","sp","df","cwb"]:
    path = f"/home/felipe/dados_vm/{city}"
    for number,filename in enumerate(os.listdir(path)):
        # files_list.append(filename)
        client.upload_file(f"/home/felipe/dados_vm/{city}/{filename}",'brbus-dataset',f'bronze/{city}/{filename}')
        print(number)

    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [ ]:
from pyspark.sql import *
from delta import *
from pyspark.sql.functions import udf, input_file_name, col

builder = SparkSession.builder \
    .appName("Salvar Delta Table no Spaces") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.jars", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.fs.s3a.access.key", "DO00LNFAEQRLTA82CJCM") \
    .config("spark.hadoop.fs.s3a.secret.key", "cOWHXzPEeesRVZSJDp/maav9CcnqYyyaOFh2HNWHcJ0") \
    .config("spark.hadoop.fs.s3a.endpoint", "nyc3.digitaloceanspaces.com")

spark = builder.getOrCreate()


INPUT_PATH = "/home/felipe/code/topicos_dados/dados/"
BRONZE_PATH = "/home/felipe/code/deltalake/lake/bronze/"
SILVER_PATH = "/home/felipe/code/deltalake/lake/silver/"


sp = spark.read.format("parquet").option("inferSchema","true").option("header","true").load(f"{INPUT_PATH}/sp_micro").withColumn("inputFiles",input_file_name())
# sp.write.mode("overwrite").format("delta").save("/home/felipe/code/topicos_dados/lake/bronze/sp/")
sp.write.mode("overwrite").format("delta").option("path","s3a://brbus-dataset/bronze/sp/").saveAsTable("bronze_sp2")

spark.stop()

In [ ]:
from pyspark.sql import *
from delta import *
from pyspark.sql.functions import udf, input_file_name, col

builder = SparkSession.builder.appName("spark") \
.config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension") \
.config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog") \
.config("spark.jars.packages","io.delta:delta-spark_2.12:3.1.0,org.apache.hadoop:hadoop-aws:3.3.4") \
.config("spark.hadoop.fs.s3a.access.key","DO00LNFAEQRLTA82CJCM") \
.config("spark.hadoop.fs.s3a.secret.key","cOWHXzPEeesRVZSJDp/maav9CcnqYyyaOFh2HNWHcJ0") \
.config("spark.hadoop.fs.s3a.endpoint", "nyc3.digitaloceanspaces.com") \

builder = SparkSession.builder.appName("spark").config("spark.hadoop.fs.s3a.access.key","DO00LNFAEQRLTA82CJCM").config("spark.hadoop.fs.s3a.secret.key","cOWHXzPEeesRVZSJDp/maav9CcnqYyyaOFh2HNWHcJ0").config("spark.hadoop.fs.s3a.endpoint", "nyc3.digitaloceanspaces.com")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

INPUT_PATH = "/home/felipe/code/topicos_dados/dados/"
BRONZE_PATH = "/home/felipe/code/deltalake/lake/bronze/"
SILVER_PATH = "/home/felipe/code/deltalake/lake/silver/"


sp = spark.read.format("parquet").option("inferSchema","true").option("header","true").load(f"{INPUT_PATH}/sp_micro").withColumn("inputFiles",input_file_name())
# sp.write.mode("overwrite").format("delta").save("/home/felipe/code/topicos_dados/lake/bronze/sp/")
sp.write.mode("overwrite").format("delta").option("path","s3a://brbus-dataset/bronze/sp/").saveAsTable("bronze_sp2")

spark.stop()

In [ ]:
from pyspark.sql import *
from delta import *
from pyspark.sql.functions import udf, input_file_name, col

builder = SparkSession.builder.appName("topicos") \
.config("spark.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
.config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension") \
.config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog").config("spark.packages","io.delta:delta-spark_2.12:3.1.0,org.apache.hadoop:hadoop-aws:3.3.4").config("spark.hadoop.fs.s3a.access.key","DO00LNFAEQRLTA82CJCM").config("spark.hadoop.fs.s3a.secret.key","cOWHXzPEeesRVZSJDp/maav9CcnqYyyaOFh2HNWHcJ0").config("spark.hadoop.fs.s3a.endpoint", "nyc3.digitaloceanspaces.com")
spark = configure_spark_with_delta_pip(builder).getOrCreate()

INPUT_PATH = "/home/felipe/code/topicos_dados/dados/"
BRONZE_PATH = "/home/felipe/code/deltalake/lake/bronze/"
SILVER_PATH = "/home/felipe/code/deltalake/lake/silver/"


sp = spark.read.format("parquet").option("inferSchema","true").option("header","true").load(f"{INPUT_PATH}/sp_micro").withColumn("inputFiles",input_file_name())
# sp.write.mode("overwrite").format("delta").save("/home/felipe/code/topicos_dados/lake/bronze/sp/")
sp.write.mode("overwrite").format("delta").option("path","s3a://brbus-dataset/bronze/sp/").saveAsTable("bronze_sp2")

spark.stop()